In [1]:
from brix import Indicator, Handler
import Simulation, US_State
import geopandas as gpd
from shapely.geometry import Point, shape
import json
import requests
import urllib
from geogrid_tools import *

ModuleNotFoundError: No module named 'Simulation'

In [ ]:
table_name='epaversion'
geom_type='block_group'

model_radius=15000
reference_radius=2500

- Create the handler
- Get the geogrid
- Identify the state
- Get the state geometry
- find the overlap
- Assign the sim, model, reference zones
- Append the grid "zones" to the actual zones (if the zones properties are updated in one indicator, will they be updated for all?)

- all the above in a world object?

In [ ]:
h=Handler(table_name)

In [ ]:
# geogrid_data=h.get_geogrid_data(include_geometries=True)

In [ ]:

# geogrid=geogrid_data.GEOGRID

In [ ]:
# geogrid=h.get_GEOGRID()
# geom = [f['geometry'] for f in geogrid['features']]
# geogrid_gdp=gpd.GeoDataFrame({'geometry':geom})

In [ ]:
get_url='https://cityio.media.mit.edu/api/table/'+table_name
with urllib.request.urlopen(get_url+'/GEOGRID') as url:
    geogrid=gpd.read_file(url.read().decode())

centroids=geogrid['geometry'].centroid
geogrid['x_centroid']=[c.x for c in centroids]
geogrid['y_centroid']=[c.y for c in centroids]
geogrid

In [ ]:
types=json.load(open('./data/default_types.json'))

In [ ]:
try:
    interactive_zone=gpd.read_file('./tables/{}/interactive.geojson'.format(table_name))
    print('using interactive zone')
except:
    interactive_zone=None
properties=init_geogrid(table_name, interactive_zone=interactive_zone, types=types)

In [ ]:
state_fips=identify_state(properties)

In [ ]:
st=US_State.US_State(state_fips, year=2018, geom_type=geom_type)
st.get_geometry()
# st.remove_non_urban_zones()
st.get_lodes_data( include=['wac', 'rac', 'od'])
st.add_lodes_cols_to_shape()
st.geom.plot()

In [ ]:
st.subset_geom_by_distance(centre_x_y=[geogrid.x_centroid.mean(), geogrid.y_centroid.mean()], 
                                   radius=reference_radius, name='reference_area')

st.subset_geom_by_distance(centre_x_y=[geogrid.x_centroid.mean(), geogrid.y_centroid.mean()], 
                                   radius=model_radius, name='model_area')

st.geom=assign_sim_area(gpd.GeoDataFrame(geogrid), st.geom)

In [ ]:
geogrid_union=gpd.GeoDataFrame(geometry=[geogrid.geometry.unary_union], data=[], crs=geogrid.crs)
sim_geoids=select_geom_by_overlap_threshold(areas_to_filter=st.geom, filter_with=geogrid_union, 
                                            area_id_col='copy_GEOID', 
                                            min_prop=0.5)
st.geom['sim_area']=False
st.geom.loc[sim_geoids, 'sim_area']=True

In [ ]:
st.geom.loc[st.geom['sim_area']].plot()

In [ ]:
areas_to_filter=st.geom
filter_with=geogrid
area_id_col='copy_GEOID'
min_prop=0.5
intersector_id_col='id'

areas_to_filter['zone_area']=areas_to_filter.geometry.area
all_intersect=gpd.overlay(areas_to_filter, filter_with, 'intersection')
all_intersect['intersect_area']=all_intersect.geometry.area
all_intersect=all_intersect[[col for col in all_intersect if not col=='zone_area']]
all_intersect=all_intersect.merge(areas_to_filter[[area_id_col, 'zone_area']], 
                                how='left', left_on=area_id_col, right_on=area_id_col)
all_intersect['prop_area']=all_intersect['intersect_area']/all_intersect['zone_area']
valid_intersect=all_intersect.loc[all_intersect['prop_area']>min_prop]
valid_ids=list(valid_intersect[area_id_col])



In [ ]:
zones=st.geom.rename(columns={col: 'area' for col in st.geom.columns if 'ALAND' in col})

In [ ]:
import os
if not os.path.isdir('tables/{}'.format(table_name)):
    os.mkdir('tables/{}'.format(table_name))

In [ ]:
zones.loc[zones['model_area']].to_file('tables/{}/zones.geojson'.format(table_name), driver='GeoJSON')

In [ ]:
simpop_df=st.lodes_to_pop_table(
    model_subset_name='model_area', sim_subset_name='sim_area')
simpop_df.to_csv('tables/{}/simpop_df.csv'.format(table_name), index=False)

In [ ]:
geogrid.to_file('tables/{}/geogrid.geojson'.format(table_name), driver='GeoJSON')

In [ ]:
assert len(zones.loc[((zones['sim_area'])&(~zones['reference_area']))])==0

In [ ]:
import matplotlib.pyplot as plt
fig, ax= plt.subplots()

zones.loc[zones['sim_area']].plot(ax=ax, color='red', alpha=0.2)
zones.loc[zones['reference_area']].plot(ax=ax, color='blue', alpha=0.2)